In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../../bar_movement/data/'

In [3]:
currency_pair = 'Usd_Cad'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback

def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['atr'] = atr(df)
df['rsi'] = rsi(df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,adx,atr,rsi
0,2021-04-01 07:10:00,1.25947,1.25956,1.25890,1.25892,1.25963,1.25973,1.25906,1.25910,1.25955,1.25964,1.25898,1.25901,145,3.467514,0.000547,39.769719
1,2021-04-01 07:15:00,1.25894,1.25942,1.25889,1.25921,1.25913,1.25960,1.25908,1.25938,1.25904,1.25951,1.25898,1.25930,106,9.863821,0.000542,45.501590
2,2021-04-01 07:20:00,1.25923,1.25925,1.25870,1.25870,1.25940,1.25942,1.25886,1.25886,1.25932,1.25934,1.25878,1.25878,85,12.163337,0.000555,38.437906
3,2021-04-01 07:25:00,1.25868,1.25879,1.25830,1.25859,1.25882,1.25895,1.25846,1.25876,1.25875,1.25887,1.25838,1.25868,130,15.265856,0.000552,37.240606
4,2021-04-01 07:30:00,1.25862,1.25869,1.25823,1.25823,1.25879,1.25886,1.25839,1.25839,1.25870,1.25878,1.25831,1.25831,104,19.576855,0.000545,33.128779


In [8]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,adx,atr
74680,2022-04-01 05:35:00,1.25016,1.25050,1.25013,1.25050,1.25038,1.25070,1.25033,1.25069,1.25027,1.25060,1.25023,1.25060,209,21.030332,0.000204
74681,2022-04-01 05:40:00,1.25049,1.25056,1.25032,1.25056,1.25073,1.25075,1.25053,1.25074,1.25061,1.25065,1.25042,1.25065,191,22.425238,0.000214
74682,2022-04-01 05:45:00,1.25055,1.25064,1.25042,1.25047,1.25075,1.25085,1.25062,1.25067,1.25065,1.25074,1.25052,1.25057,115,23.858273,0.000220
74683,2022-04-01 05:50:00,1.25046,1.25086,1.25046,1.25061,1.25068,1.25109,1.25068,1.25081,1.25057,1.25097,1.25057,1.25071,140,25.441157,0.000241
74684,2022-04-01 05:55:00,1.25061,1.25098,1.25059,1.25088,1.25082,1.25119,1.25080,1.25109,1.25072,1.25108,1.25070,1.25098,239,27.410257,0.000258


In [9]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [10]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, adx_cutoff, atr_multiplier, sl):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1
    lookforward = -1
    pip_movement /= divider

    for i in range(12, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - 12:i], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - 12:i], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])
        adx_values = list(df.loc[df.index[i - lookback:i + lookforward], 'adx'])
        atr_values = list(df.loc[df.index[i - lookback:i + lookforward], 'atr'])

        if atr_multiplier is None:
            buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
            sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        else:
            buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= atr_values[0] * n_bars * atr_multiplier
            sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= atr_values[0] * n_bars * atr_multiplier

        if buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= 0.35 * abs(mid_high1 - mid_low1)

        if sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= 0.35 * abs(mid_high1 - mid_low1)

        moving_enough = all([adx_val > adx_cutoff for adx_val in adx_values])

        buy_signal = buy_signal and moving_enough
        sell_signal = sell_signal and moving_enough

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal and curr_ask_low <= curr_ask_open:
                open_price = float(curr_ask_open)
                atr1 = df.loc[df.index[i - 1], 'atr']

                if sl == 'hourly':
                    stop_loss = lowest_low

                elif sl =='atr':
                    stop_loss = open_price - atr1

                else:
                    stop_loss = min([open_price - atr1, min(mid_lows[-n_bars - 1:])])

                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal and curr_bid_high >= curr_bid_open:
                open_price = float(curr_bid_open)
                atr1 = df.loc[df.index[i - 1], 'atr']
                
                if sl == 'hourly':
                    stop_loss = highest_high

                elif sl =='atr':
                    stop_loss = open_price + atr1

                else:
                    stop_loss = max([open_price + atr1, max(mid_highs[-n_bars - 1:])])

                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
# risk_reward_ratio_vals = [1.0, 1.5, 2.0]
# spread_cutoffs = [0.10, 0.20, 0.25]
# n_bars_vals = [3]
# pip_movement_vals = [20, 30]
# adx_cutoffs = [0, 25, 30]
# atr_multipliers = [None, 1.0, 1.5, 2.0]
# sl_vals = ['hourly', 'atr', 'mix']

risk_reward_ratio_vals = [2.0]
spread_cutoffs = [0.10]
n_bars_vals = [3]
pip_movement_vals = [20]
adx_cutoffs = [0]
atr_multipliers = [None]
sl_vals = ['mix']

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for adx_cutoff in adx_cutoffs:
                    for atr_multiplier in atr_multipliers:
                        for sl in sl_vals:
                            all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, adx_cutoff, atr_multiplier, sl))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_adx_cutoff = None
best_atr_multiplier = None
best_sl = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, adx_cutoff, atr_multiplier, sl in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, adx_cutoff, atr_multiplier, sl)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'adx_cutoff': adx_cutoff, 'atr_multiplier': atr_multiplier, 'sl': sl})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_adx_cutoff = adx_cutoff
        best_atr_multiplier = atr_multiplier
        best_sl = sl

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1

1320.3639800000158
Num buys: 39
Num sells: 29
Num trades: 68
Num wins: 30
Num losses: 38
Win streak: 4
Loss streak: 5
Avg pips risked: 0.00275852941176472
Remaining runs: 0
Best reward so far: 1320.3639800000158



In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best sl: ' + str(best_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1088.7723900000028
Best risk/reward ratio: 2.0
Best spread: 0.1
Best n bars val: 3
Best pip movement val: 20
Best adx cutoff: 0
Best atr multiplier: None
Best sl: mix
-----------------------
Top results:
{'reward': 1006, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'adx_cutoff': 0, 'atr_multiplier': None, 'sl': 'mix'}
{'reward': 1087, 'ratio': 2.0, 'spread': 0.2, 'n_bars': 3, 'pip_movement': 20, 'adx_cutoff': 0, 'atr_multiplier': None, 'sl': 'mix'}
{'reward': 1056, 'ratio': 1.0, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'adx_cutoff': 0, 'atr_multiplier': None, 'sl': 'hourly'}
{'reward': 1006, 'ratio': 1.5, 'spread': 0.2, 'n_bars': 3, 'pip_movement': 20, 'adx_cutoff': 0, 'atr_multiplier': None, 'sl': 'mix'}
{'reward': 1054, 'ratio': 1.0, 'spread': 0.2, 'n_bars': 3, 'pip_movement': 20, 'adx_cutoff': 0, 'atr_multiplier': None, 'sl': 'hourly'}
{'reward': 688, 'ratio': 1.5, 'spread': 0.2, 'n_bars': 3, 'pip_movement'